# Recomendador de artigos - Open Alex e GPT

In [2]:
import requests
import pandas as pd
from datetime import datetime,timedelta,date
import os
import pyarrow.parquet as pq


pd.options.display.max_columns = 999

In [3]:
def coletar_dados_e_salvar(data_inicial,data_fim):
    
    cursor = '*'
            
    contador = 1
    contador_erro = 0
    
    diretorio = f'datasets_{data_inicial}_to_{data_fim}'
    
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
    
    while cursor != None:
        url = f'https://api.openalex.org/works?filter=from_publication_date:{data_inicial},to_publication_date:{data_fim},type:Article&per-page=200&cursor={cursor}'

        
        try:
            requisicao = requests.get(url)
            pagina_com_resultados = requisicao.json()
            
        except (requests.exceptions.RequestException, ValueError) as e:
            contador_erro += 1 
            print(f"Erro na página {contador} (Erro {contador_erro}): {e}")
            continue 
        
        resultados = pagina_com_resultados.get('results', [])
        
        df = pd.DataFrame(resultados)
        parquet_arquivo = os.path.join(diretorio, f'registros_{data_inicial}_to_{data_fim}_{contador}.parquet')
        df.to_parquet(parquet_arquivo, index=False)
        
        cursor = pagina_com_resultados['meta'].get('next_cursor')
        
        contador += 1
    
    print(f"Total de erros: {contador_erro}")

In [4]:
def concatenar_arquivos_parquet(folder_path):

    dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.parquet'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_parquet(file_path)
            dataframes.append(df)

    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    df_concatenado.to_parquet('df_concatenado.parquet',index=False)
    

## CONTINUAR

In [6]:
def extrair_concepts_scores(df):
    concept_data = []

    for concepts in df['concepts']:
        concept_scores = {}
        for concept in concepts:
            if concept['level'] == 0:
                concept_scores[concept['display_name']] = concept['score']
            elif concept['level'] == 1 and concept['score'] > 0.7:
                concept_scores[concept['display_name']] = concept['score']
        concept_data.append(concept_scores)

    df_concepts = pd.DataFrame(concept_data, index=df.index).fillna(0).round(4)

    df_final = pd.concat([df, df_concepts], axis=1)

    return df_final

In [7]:
def get_source_name(primary_location):
    if primary_location is None:
        return None
    source_info = primary_location.get('source', None)
    
    if source_info is None:
        return None
    display_name = source_info.get('display_name', None)
    
    return display_name



def transformar_df(df):
    
    df = df.loc[:,['doi','title','publication_date','created_date','primary_location','open_access','concepts','cited_by_count']]
    
    df_filtrado = df.copy()
    
    df_filtrado.loc[:,'source_title'] = df_filtrado['primary_location'].apply(get_source_name)
    
    df_filtrado.loc[:,'open_access_status'] = df_filtrado['open_access'].apply(lambda x: x.get('is_oa',None))
    
    df_filtrado = extrair_concepts_scores(df_filtrado)
    
    df_filtrado.drop(columns=['primary_location','open_access','concepts'],inplace=True)
    
    return df_filtrado

In [8]:
def ler_e_transformar_arquivos_diretorio(diretorio_input, diretorio_output):
    
    if not os.path.exists(diretorio_output):
        os.makedirs(diretorio_output)
    
    arquivos = [arquivo for arquivo in os.listdir(diretorio_input) if arquivo.endswith(".parquet")]
    
    try:
        for arquivo_parquet in arquivos:
            caminho_input = os.path.join(diretorio_input, arquivo_parquet)
            df_original = pd.read_parquet(caminho_input)  
            df_processado = transformar_df(df_original) 
            
            output_file = os.path.splitext(arquivo_parquet)[0] + ".parquet"
            caminho_output = os.path.join(diretorio_output, output_file)
            
            df_processado.to_parquet(caminho_output, index=False)

    except Exception as error:
        print(arquivo_parquet,error)

In [9]:
def concatenar_arquivos_parquet(diretorio_input):
    dfs = []

    for arquivo in os.listdir(diretorio_input):
        if arquivo.endswith('.parquet'):
            file_path = os.path.join(diretorio_input, arquivo)
            df_parquet = pd.read_parquet(file_path)
        
            dfs.append(df_parquet)

    df_concatenado = pd.concat(dfs, ignore_index=True)

    return df_concatenado

In [8]:
data_inicial = '2023-09-23'
data_final = '2023-09-30'

#coletar_dados_e_salvar(data_inicial,data_final)

In [10]:
diretorio_input = 'datasets_2023-09-23_to_2023-09-30'
diretorio_output = 'datasets_2023-09-23_to_2023-09-30_processados'

In [11]:
ler_e_transformar_arquivos_diretorio(diretorio_input,diretorio_output)

In [12]:
df_concatenado = concatenar_arquivos_parquet(diretorio_output)

In [13]:
df_concatenado.iloc[:,7:] = df_concatenado.iloc[:,7:].fillna(0)

In [ ]:
df_concatenado

In [ ]:
concepts = df_concatenado.columns[7:]

def create_concept_column(row):
    concepts_list = [concept for concept in concepts if row[concept] > 0]
    return ';'.join(concepts_list)

df_concatenado['Concepts'] = df_concatenado.apply(create_concept_column, axis=1)

In [18]:
df_concatenado['Concepts'].value_counts(normalize=True)

Concepts
Medicine                                                                  0.145206
Biology                                                                   0.049516
Medicine;Biology                                                          0.048023
Computer science                                                          0.045146
Chemistry;Materials science                                               0.025539
                                                                            ...   
Psychology;Engineering;Chemistry;Sociology;Art;History                    0.000014
Medicine;Political science;Computer science;Business;Biology              0.000014
Engineering;Business;Environmental science;Geography;Materials science    0.000014
Computer science;Geography;Art;History                                    0.000014
Medicine;Psychology;Political science;Mathematics;Geography               0.000014
Name: proportion, Length: 2969, dtype: float64

In [ ]:
df_concatenado.to_parquet('df_concatenado.parquet')

___

In [ ]:
df_concatenado = pd.read_parquet('df_concatenado.parquet')

In [ ]:
df_concatenado = pd.read_parquet('df_concatenado.parquet')


In [ ]:
df_concatenado['Concepts'].value_counts(normalize=True)

In [ ]:
concept = 'Statistics'

df_concatenado.query('`Statistics` > 0 ')[
    ['doi','title','publication_date','cited_by_count',concept]
    ].sort_values(by='Statistics',ascending=False)